# Part 8 : Pipeline

In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime as dt
import calendar as cal

In [98]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [99]:
dataset=pd.read_csv('C:\\Users\\DELL\\Desktop\\ADS\\Appliances-energy-prediction-data-master\\energydata_complete.csv',parse_dates=['date'])

In [100]:
dataset.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [101]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
date           19735 non-null datetime64[ns]
Appliances     19735 non-null int64
lights         19735 non-null int64
T1             19735 non-null float64
RH_1           19735 non-null float64
T2             19735 non-null float64
RH_2           19735 non-null float64
T3             19735 non-null float64
RH_3           19735 non-null float64
T4             19735 non-null float64
RH_4           19735 non-null float64
T5             19735 non-null float64
RH_5           19735 non-null float64
T6             19735 non-null float64
RH_6           19735 non-null float64
T7             19735 non-null float64
RH_7           19735 non-null float64
T8             19735 non-null float64
RH_8           19735 non-null float64
T9             19735 non-null float64
RH_9           19735 non-null float64
T_out          19735 non-null float64
Press_mm_hg    19735 non-null float64
RH_out        

In [102]:
dataset.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [103]:
dataset['Year'] = pd.DatetimeIndex(dataset['date']).year
dataset['Month'] = pd.DatetimeIndex(dataset['date']).month
dataset['Monthname'] = dataset['date'].dt.month.apply(lambda x : cal.month_name[x])
dataset['Day']=pd.DatetimeIndex(dataset['date']).day
dataset['Hour']=pd.DatetimeIndex(dataset['date']).hour
dataset['Perday']=dataset['date'].dt.date
dataset['Time']=dataset['date'].dt.time
dataset['DayoftheWeek']=dataset['date'].dt.weekday_name
dataset['WeekStatus'] = dataset['date'].dt.dayofweek
dataset['Weekendstatus'] = np.where(dataset['WeekStatus'] < 5, 0, 1)

#To Calculate NSM
datenum=dataset['Perday'].unique()
lennsm = len(datenum)+1
print("len",lennsm)
data=[]
for j in range (1,lennsm):
    if (j==1):
        for i in range (0,42):
            if (i==0):
                a = 61200
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
    elif (j > 1 ):
        for i in range (0,144):
            if (i==0):
                a = 0
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
df1 = pd.DataFrame({'data_converted' : data})
dfc1 = df1[:len(dataset)]
dataset['NSM'] = dfc1
###################

len 139


In [104]:
df_day = pd.get_dummies(dataset['DayoftheWeek'])
dataset = dataset.join(df_day)

In [105]:
#Deleting ome of the columns from the dataset as they are not needed pr their purpose is done.

# Monday column is deleted as the result can be derived by the other six columns of the week
del dataset['Monday']

# WeekStatus column is deleted as the column was only made to derive weekend status column
# As we needed the categorical values for weekday and weekend
del dataset['WeekStatus'] 

# DayoftheWeek column is deleted as there are separate columns for each of the weekday
del dataset['DayoftheWeek']
del dataset['Time'] 

# Perday column was deleted as it was only needed for NSM
del dataset['Perday'] 

In [106]:
del dataset['date']
del dataset['rv1']
del dataset['rv2']
del dataset['Monthname']
del dataset['Year']

In [107]:
dataset.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,Day,Hour,Weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,16.057411,11.502002,0.277274,42907.129465,0.144160,0.138637,0.138637,0.145934,0.145934,0.145934
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,8.450998,6.921953,0.447664,24940.020831,0.351261,0.345576,0.345576,0.353049,0.353049,0.353049
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,9.000000,6.000000,0.000000,21600.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,16.000000,12.000000,0.000000,43200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,23.000000,17.000000,1.000000,64200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,31.000000,23.000000,1.000000,85800.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [108]:
dataset.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,Day,Hour,Weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,11,17,0,61200,0,0,0,0,0,0
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,11,17,0,61800,0,0,0,0,0,0
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,11,17,0,62400,0,0,0,0,0,0
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,11,17,0,63000,0,0,0,0,0,0
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,11,17,0,63600,0,0,0,0,0,0


In [109]:
X=dataset[['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'Month','Hour','Day','Weekendstatus','NSM',
      'Friday','Saturday','Sunday','Thursday','Tuesday','Wednesday']]

In [110]:
y=dataset['Appliances']

In [111]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression

In [112]:
#train_test_split : Basically passing the x and y data and then specifying test size
#Split arrays or matrices into random train and test subsets

In [113]:
# In this case, the attribue random_state basically refers to the random split of the data between train set and test set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)
lm=LinearRegression()
lm.fit(X_train,y_train)
print(lm.intercept_)
print(lm.coef_)
X_train.columns

-2.8725637530753403
[ 1.87163083e+00 -5.63536051e+00  1.54069958e+01 -1.81293023e+01
 -1.37455577e+01  2.52457081e+01  5.67791133e+00  5.99931309e-01
 -1.59492240e+00 -1.63222045e-01  3.29654692e-03  7.33789030e+00
 -7.65859523e-04 -3.22970473e-01 -1.12567166e+00  8.57809477e+00
 -4.21535310e+00 -1.02009934e+01 -4.13167482e-01 -7.82511412e+00
  7.57057328e-02 -1.66094927e-01  1.40657299e+00  1.40089497e-01
  1.99069073e+00 -9.68569290e+00 -4.35478717e+00 -4.92337803e-03
  1.94381618e+00  1.46341202e-03  9.69795297e+00  1.00938177e+01
 -8.15000155e+00 -9.82273020e+00 -1.27760765e+01 -1.02777991e+01]


Index(['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5',
       'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out',
       'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint',
       'Month', 'Hour', 'Day', 'Weekendstatus', 'NSM', 'Friday', 'Saturday',
       'Sunday', 'Thursday', 'Tuesday', 'Wednesday'],
      dtype='object')

### Predictions

In [116]:
from sklearn import metrics

In [123]:
error_metric = pd.DataFrame({'r2_train': [],
                            'r2_test': [],
                            'rmse_train':[], 
                            'rmse_test': [],
                            'mae_train': [],
                            'mae_test':[],
                            'mse_train':[],
                            'mse_test':[]})



def calc_error_metric(modelname, model, X_train_scale, y_train, X_test_scale, y_test):
    global error_metric
    y_train_predicted = model.predict(X_train)
    y_test_predicted = model.predict(X_test)
    
    mae_train=mean_absolute_error(y_train, y_train_predicted)
    mae_test=mean_absolute_error(y_test, y_test_predicted)
    
    
    mse_train=mean_squared_error(y_train, y_train_predicted)
    mse_test=mean_squared_error(y_test, y_test_predicted)
    
    rmse_train=np.sqrt(mean_squared_error(y_train, y_train_predicted))
    rmse_test=np.sqrt(mean_squared_error(y_test, y_test_predicted))
   
    r2_train=r2_score(y_train, y_train_predicted)
    r2_test=r2_score(y_test, y_test_predicted)
   
    
    df_local = pd.DataFrame({'r2_train': [r2_train],
                            'r2_test': [r2_test],
                            'rmse_train':[rmse_train], 
                            'rmse_test': [rmse_test],
                            'mae_train': [mae_train],
                            'mae_test': [mae_test],
                            'mse_train':[mse_train],
                            'mse_test':[mse_test]})
        
    error_metric = pd.concat([error_metric, df_local])
    return error_metric
    
    

In [124]:
pipe_lr = Pipeline([('scl', StandardScaler()),('clf', LinearRegression(normalize=True))])
grid_params_lr =[{}]
gs_lr = GridSearchCV(estimator=pipe_lr, param_grid=grid_params_lr, cv=10) 
gs_lr.fit(X_train, y_train)
em=calc_error_metric('Regression', gs_lr, X_train, y_train, X_test, y_test)
print('Regression completed')

Regression completed


In [125]:
em

,r2_train,r2_test,rmse_train,rmse_test,mae_train,mae_test,mse_train,mse_test
0,0.174365,0.175189,94.789112,89.16968,53.660834,51.968165,8984.975747,7951.231908
